## Hold-up Tank Model

Using fmdtools to simulate hazards in a system with human-component interactions, including:

- human-induced failure modes
- human responses to component failure modes
- joint human-component failure modes

The system to model is in `tank_model.py`

In [1]:
import fmdtools.sim.propagate as propagate
import fmdtools.analyze as an
from tank_model import Tank
from fmdtools.define import SampleApproach

ImportError: cannot import name 'SampleApproach' from 'fmdtools.define' (c:\users\dhulse\documents\github\fmdtools\fmdtools\define\__init__.py)

### Verifying the nominal state:

In the nominal state, no change in system state should occur and the tank level should remain at 5.

In [2]:
mdl = Tank()
resgraph, mdlhist = propagate.nominal(mdl, desired_result='fxnflowgraph')

In [3]:
mdlhist.keys()

In [4]:
an.graph.show(resgraph)

In [5]:
fig, axs = an.plot.mdlhists(mdlhist)

### What happens under component faults?


Here we model a leak of the tank. To compensate for this leak, the operator opens the first valve to a higher setting, maintaining the level of the tank

In [6]:
resgraph, mdlhist = propagate.one_fault(mdl,'Store_Water','Leak', time=3, desired_result='fxnflowgraph')

fig, ax = an.plot.mdlhists(mdlhist, title="Tank response to leak at t=3", time_slice=[3], fxnflowvals={'Store_Water':['level', 'net_flow'], 'Tank_Sig':['indicator'], 'Valve1_Sig':['action']}, legend_loc=False)
fig, ax = an.graph.show(resgraph,faultscen='Leak', time=3)


### What about human-induced faults?

Here we evaluate what happens if the operator thinks they see a low or high indicator and takes those given actions.

Note that in these cases, because of the indicator/procedures, the operators are able to correct for the fault.

In [7]:
resgraph, mdlhist = propagate.one_fault(mdl,'Human','FalseDetection_low', time=3, desired_result='fxnflowgraph')

fig, ax= an.plot.mdlhists(mdlhist, title='FalseDetection_low', time_slice=[3], fxnflowvals={'Store_Water':['level', 'net_flow'], 'Tank_Sig':['indicator'], 'Valve1_Sig':['action']}, legend_loc=False)
fig = an.graph.show(resgraph,faultscen='FalseDetection_low', time=3)

In [8]:
resgraph, mdlhist = propagate.one_fault(mdl,'Human','FalseDetection_high', time=3, desired_result='fxnflowgraph')

fig, ax= an.plot.mdlhists(mdlhist, title='FalseDetection_high', time_slice=[3], fxnflowvals={'Store_Water':['level', 'net_flow'], 'Tank_Sig':['indicator'], 'Valve1_Sig':['action']}, legend_loc=False)
fig = an.graph.show(resgraph,faultscen='FalseDetection_high', time=3)

### Evaluating Joint Component-Human fault modes 

To understand where the risks of failure are, we need to find the scenarios, that, with the modelled human controls, still lead to failures. To assess this, we develop a sample approach.

In [9]:
#app_full = SampleApproach(mdl)
#endclasses, mdlhists = fp.run_approach(mdl, app_full)

Here we consider all single and joint-fault scenarios in the set of simulations to see which ones lead to failure:

In [10]:
app_joint_faults = SampleApproach(mdl, faults='all', jointfaults={'faults': 2})
endclasses, mdlhists = propagate.approach(mdl, app_joint_faults)

In [11]:
fmea_tab = an.tabulate.simplefmea(endclasses)
fmea_tab

Next, we can filter out non-failures and sort by the failures with the highest expected cost (though rate would give the same results here)

In [12]:
failure_tab = fmea_tab[fmea_tab['cost'] > 1]
failure_tab.sort_values('expected cost', ascending = False)

One of the top modes is a joint human-component failure mode. Let's see what happens in this case:

In [13]:
scenhists ={'nominal':mdlhists['nominal'], 'faulty':mdlhists['Guide_Water_Out: Leak, Human: FalseDetection_high, t=0']}
fig, axs = an.plot.mdlhists(scenhists)

In this case, there is a leak, but the operator cannot turn the valve, resulting in the tank filling too high, which is a failure.

To consider the leak again, we can see what happens when the leak is not detected:

In [14]:
scenhists ={'nominal':mdlhists['nominal'], 'faulty':mdlhists['Guide_Water_Out: Leak, Human: FalseDetection_low, t=0']}
fig, axs = an.plot.mdlhists(scenhists)

In this case, there is a leak, but it is not caugh, resulting in a failure again.

### Testing different reaction times

The model set up in `tank_model` is parameterized by the reaction time of the operator. As a result, we can assess how long or short reaction times affect the given scenarios.

In [15]:
mdl_long_reaction_time = Tank(params={'reacttime':10, "store_tstep":1.0})

In this case, we will show the affect of reaction time on the operator's ability to catch a leak.

In [16]:
resgraph, mdlhist = propagate.one_fault(mdl_long_reaction_time,'Store_Water','Leak', time=3, desired_result='fxnflowgraph')

fig, axs = an.plot.mdlhists(mdlhist, title = "Response of Tank to Leak", time_slice=3, fxnflowvals={'Store_Water':['level', 'net_flow'], 'Tank_Sig':['indicator'], 'Valve1_Sig':['action']}, legend_loc=False)
fig, ax = an.graph.show(resgraph,faultscen='Leak', time=3)


As shown, the operator does not respond in time, resulting in the tank draining all the way, a failure state. We can further use the `propagate.approach` function to compare the number of scenarios caught in this system compared with the other.